In [ ]:
import pandas as pd
df = pd.read_csv("/content/EEG_data_classification_task.csv")


In [ ]:
df.shape

(12811, 12)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12811 entries, 0 to 12810
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Attention           12811 non-null  float64
 1   Mediation           12811 non-null  float64
 2   Raw                 12811 non-null  float64
 3   Delta               12811 non-null  float64
 4   Theta               12811 non-null  float64
 5   Alpha1              12811 non-null  float64
 6   Alpha2              12811 non-null  float64
 7   Beta1               12811 non-null  float64
 8   Beta2               12811 non-null  float64
 9   Gamma1              12811 non-null  float64
 10  Gamma2              12811 non-null  float64
 11  user-definedlabeln  12811 non-null  int64  
dtypes: float64(11), int64(1)
memory usage: 1.2 MB


In [ ]:
df.head()

,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,user-definedlabeln
0,56.0,43.0,278.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0
1,40.0,35.0,-50.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,0
2,47.0,48.0,101.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,25354.0,0
3,47.0,57.0,-5.0,2012240.0,129350.0,61236.0,17084.0,11488.0,62462.0,49960.0,33932.0,0
4,44.0,53.0,-8.0,1005145.0,354328.0,37102.0,88881.0,45307.0,99603.0,44790.0,29749.0,0


In [ ]:
X = df.drop('user-definedlabeln' ,axis=1)
Y = df['user-definedlabeln']

In [ ]:
X.shape,Y.shape

((12811, 11), (12811,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.3, random_state=7)

In [ ]:
from sklearn import preprocessing
x = X_train.values #returns a numpy array
std_scaler = preprocessing.StandardScaler()
std_scaler.fit(x)
x_scaled_std = std_scaler.transform(x)
X_train_std = pd.DataFrame(x_scaled_std,columns=X_train.columns)

In [ ]:
x = X_test.values
x_scaled_std = std_scaler.transform(x)
X_test_std = pd.DataFrame(x_scaled_std,columns=X_test.columns)

In [ ]:
X_train_std.describe()

,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2
count,8.967000e+03,8.967000e+03,8.967000e+03,8.967000e+03,8.967000e+03,8.967000e+03,8.967000e+03,8.967000e+03,8.967000e+03,8.967000e+03,8.967000e+03
mean,1.752684e-16,-2.458475e-16,-9.100189e-18,7.745684e-17,-3.825175e-17,-2.777724e-17,-2.772153e-17,1.847276e-17,-6.410062e-17,-2.578387e-18,2.217475e-17
std,1.000056e+00,1.000056e+00,1.000056e+00,1.000056e+00,1.000056e+00,1.000056e+00,1.000056e+00,1.000056e+00,1.000056e+00,1.000056e+00,1.000056e+00
min,-1.774155e+00,-2.070433e+00,-3.490151e+00,-9.491271e-01,-6.877586e-01,-5.683341e-01,-5.703145e-01,-6.540752e-01,-4.683774e-01,-3.617228e-01,-3.997891e-01
25%,-6.540201e-01,-4.389149e-01,-1.361893e-01,-7.957770e-01,-5.768700e-01,-4.741740e-01,-4.519561e-01,-4.891606e-01,-3.790916e-01,-3.126682e-01,-3.400079e-01
50%,7.837613e-02,1.784161e-01,-5.374190e-02,-3.330853e-01,-3.506169e-01,-3.279047e-01,-3.142587e-01,-3.090353e-01,-2.767480e-01,-2.430099e-01,-2.593695e-01
75%,6.815259e-01,7.075571e-01,3.530133e-02,4.942404e-01,1.357941e-01,4.529346e-02,2.252441e-02,7.994845e-02,-4.138991e-02,-6.393683e-02,-4.818756e-02
max,2.534057e+00,2.339075e+00,3.262294e+00,5.264632e+00,1.171455e+01,1.821778e+01,1.429648e+01,2.206736e+01,1.951771e+01,2.348854e+01,3.684235e+01


## Feature Selection

## RFR


In [ ]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model_dt = DecisionTreeClassifier()

In [ ]:
rfe = RFE(estimator=model_dt, n_features_to_select=1, step=1)

In [ ]:
rfe.fit(X_train,Y_train)

RFE(estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                     criterion='gini', max_depth=None,
                                     max_features=None, max_leaf_nodes=None,
                                     min_impurity_decrease=0.0,
                                     min_impurity_split=None,
                                     min_samples_leaf=1, min_samples_split=2,
                                     min_weight_fraction_leaf=0.0,
                                     presort='deprecated', random_state=None,
                                     splitter='best'),
    n_features_to_select=1, step=1, verbose=0)

In [ ]:
for rank, col in sorted(zip(rfe.ranking_,X.columns)):
    print(rank, col)

1 Delta
2 Gamma2
3 Gamma1
4 Beta2
5 Theta
6 Beta1
7 Alpha2
8 Alpha1
9 Raw
10 Mediation
11 Attention


In [ ]:
X_train_rfe = X_train.loc[:, ['Delta','Gamma2','Gamma1','Beta2','Theta']]
X_test_rfe = X_test.loc[:, ['Delta','Gamma2','Gamma1','Beta2','Theta']]

## PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(0.95) 
pca.fit(X_train)
X_train_new = pca.transform(X_train)
X_test_new = pca.transform(X_test)
print(X_train_new.shape)
X_train_new[:5]

(8967, 2)


array([[  96113.24882781,   97341.62777643],
       [1049768.14889772, -161287.35566594],
       [-259947.61630855, -102312.87726472],
       [ 348756.89197366,  240439.47880639],
       [ 866547.64483574, -166765.78127545]])

## LASSO

In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=1.0)
lasso.fit(X_train, Y_train)
lasso.coef_

array([-0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  7.64295155e-08,
        1.64646592e-07,  1.87152173e-07,  1.34910873e-07,  2.43677711e-07,
       -3.39818718e-07, -3.55209543e-07,  9.59030519e-07])

In [ ]:
selected_features = X_train.columns[lasso.coef_ > 0 ]

In [ ]:
selected_features

Index(['Delta', 'Theta', 'Alpha1', 'Alpha2', 'Beta1', 'Gamma2'], dtype='object')

In [ ]:
X_train_lasso = X_train.loc[:,['Delta', 'Theta', 'Alpha1', 'Alpha2', 'Beta1', 'Gamma2']]
X_test_lasso = X_test.loc[:,['Delta', 'Theta', 'Alpha1', 'Alpha2', 'Beta1', 'Gamma2']]

## RIDGE 

In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=1000)
ridge.fit(X_train,Y_train)
ridge.coef_

array([-2.57690646e-03,  1.08700017e-03, -3.23638005e-06,  6.90485488e-08,
        1.50749600e-07,  1.29325847e-07,  7.32763518e-08,  2.80501436e-07,
       -3.05875079e-07, -4.19581642e-07,  8.76052646e-07])

In [ ]:
Y_pred_ridge = ridge.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test,Y_pred_ridge)

0.23991371604603673

### Naive Bayes


Gaussian NB 

Multinomial NB

Complement NB 

Bernoulli NB

Categorical NB 

In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB,ComplementNB, BernoulliNB,CategoricalNB

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train_std, Y_train)
Y_pred_gnb = gnb.predict(X_test_std)

In [ ]:
def evaluate_clf(Y_test, Y_pred):
  from sklearn.metrics import precision_recall_fscore_support
  p,r,f,s = precision_recall_fscore_support(Y_test,Y_pred)
  return p,r,f,s

In [ ]:
print("Gaussian NB")
print(evaluate_clf(Y_test, Y_pred_gnb))

Gaussian NB
(array([0.51648   , 0.63699583]), array([0.8608    , 0.23260538]), array([0.6456    , 0.34077381]), array([1875, 1969]))


In [ ]:
gnb = GaussianNB()
gnb.fit(X_train_rfe, Y_train)
Y_pred_gnbr = gnb.predict(X_test_rfe)

In [ ]:
print(evaluate_clf(Y_test, Y_pred_gnbr))

(array([0.53181019, 0.63763764]), array([0.80693333, 0.32351447]), array([0.64110169, 0.42924528]), array([1875, 1969]))


# PCA Feature

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train_new, Y_train)
Y_pred_gnbp = gnb.predict(X_test_new)

In [ ]:
print(evaluate_clf(Y_test, Y_pred_gnbp))

(array([0.52889359, 0.62823062]), array([0.80053333, 0.32097511]), array([0.6369616 , 0.42487395]), array([1875, 1969]))


# LASSO

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train_lasso, Y_train)
Y_pred_gnbl = gnb.predict(X_test_lasso)

In [ ]:
print(evaluate_clf(Y_test, Y_pred_gnbl))

(array([0.5137354 , 0.63367799]), array([0.86773333, 0.21787709]), array([0.64537882, 0.32426304]), array([1875, 1969]))


## BernoulliNB

In [ ]:
bnb = BernoulliNB()
bnb.fit(X_train_std, Y_train) 
Y_pred_bnb = bnb.predict(X_test_std)

In [ ]:
print("Burnoilli NB")
print(evaluate_clf(Y_test, Y_pred_bnb))

Burnoilli NB
(array([0.55198238, 0.60482846]), array([0.66826667, 0.48349416]), array([0.60458384, 0.53739769]), array([1875, 1969]))


## RFE Feature

In [ ]:
bnb = BernoulliNB()
bnb.fit(X_train_rfe, Y_train) 
Y_pred_bnbr = bnb.predict(X_test_rfe)

In [ ]:
print(evaluate_clf(Y_test, Y_pred_bnbr))

(array([0.        , 0.51222685]), array([0., 1.]), array([0.       , 0.6774471]), array([1875, 1969]))


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## PCA Feature

In [ ]:
bnb = BernoulliNB()
bnb.fit(X_train_new, Y_train) 
Y_pred_bnbp = bnb.predict(X_test_new)

In [ ]:
print(evaluate_clf(Y_test, Y_pred_bnbp))

(array([0.56537313, 0.57215307]), array([0.50506667, 0.63026917]), array([0.53352113, 0.59980667]), array([1875, 1969]))


LASSO Feature

In [ ]:
bnb = BernoulliNB()
bnb.fit(X_train_lasso, Y_train) 
Y_pred_bnbl = bnb.predict(X_test_lasso)

In [ ]:
print(evaluate_clf(Y_test, Y_pred_bnbl))

(array([0.        , 0.51222685]), array([0., 1.]), array([0.       , 0.6774471]), array([1875, 1969]))


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
cnb = ComplementNB()
cnb.fit(X_train_std, Y_train) 
Y_pred_cnb =cnb.predict(X_test_std)

ValueError: ignored

## Support Vector Machine Classifier

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc = SVC()
svc.fit(X_train_std, Y_train) 
Y_pred_svc = svc.predict(X_test_std)

In [ ]:
print("SVM classifier")
print(evaluate_clf(Y_test, Y_pred_svc))

SVM classifier
(array([0.5997999 , 0.63360434]), array([0.63946667, 0.59370239]), array([0.61899845, 0.61300472]), array([1875, 1969]))


## RFE Feature

In [ ]:
svc = SVC()
svc.fit(X_train_rfe, Y_train) 
Y_pred_svcr = svc.predict(X_test_rfe)

In [ ]:
print(evaluate_clf(Y_test, Y_pred_svcr))

(array([0.57207207, 0.60346696]), array([0.6096    , 0.56576943]), array([0.59024012, 0.58401048]), array([1875, 1969]))


### PCA Feature

In [ ]:
svc = SVC()
svc.fit(X_train_new, Y_train) 
Y_pred_svca = svc.predict(X_test_new)

In [ ]:
print(evaluate_clf(Y_test, Y_pred_svcr))

(array([0.57207207, 0.60346696]), array([0.6096    , 0.56576943]), array([0.59024012, 0.58401048]), array([1875, 1969]))


## LASSO Feature

In [ ]:
svc = SVC()
svc.fit(X_train_lasso, Y_train) 
Y_pred_svcl = svc.predict(X_test_lasso)

In [ ]:
print(evaluate_clf(Y_test, Y_pred_svcl))

(array([0.56945156, 0.59640782]), array([0.59253333, 0.57338751]), array([0.5807632 , 0.58467115]), array([1875, 1969]))


## Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train_std, Y_train) 
Y_pred_dt = dt.predict(X_test_std)

In [ ]:
print("DT classifier")
print(evaluate_clf(Y_test, Y_pred_dt))

DT classifier
(array([0.57524537, 0.5920398 ]), array([0.56266667, 0.6043677 ]), array([0.56888649, 0.59814024]), array([1875, 1969]))


## RFE Feature

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train_rfe, Y_train) 
Y_pred_dtr = dt.predict(X_test_rfe)

In [ ]:
print(evaluate_clf(Y_test, Y_pred_dtr))

(array([0.55046372, 0.56936847]), array([0.53813333, 0.58151346]), array([0.54422869, 0.57537688]), array([1875, 1969]))


## PCA Feature

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train_new, Y_train) 
Y_pred_dtp = dt.predict(X_test_new)

In [ ]:
print(evaluate_clf(Y_test, Y_pred_dtp))

(array([0.51061513, 0.53313403]), array([0.50026667, 0.54342306]), array([0.50538793, 0.53822938]), array([1875, 1969]))


# LASSO Feature

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train_lasso, Y_train) 
Y_pred_dtl = dt.predict(X_test_lasso)

In [ ]:
print(evaluate_clf(Y_test, Y_pred_dtl))

(array([0.54393305, 0.56780538]), array([0.55466667, 0.5571356 ]), array([0.54924743, 0.56241989]), array([1875, 1969]))


## Random forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train_std, Y_train) 
Y_pred_rf = rf.predict(X_test_std)

In [ ]:
print("RF classifier")
print(evaluate_clf(Y_test, Y_pred_rf))

RF classifier
(array([0.6493576 , 0.66497976]), array([0.64693333, 0.66734383]), array([0.6481432, 0.6661597]), array([1875, 1969]))


# RFE Feature

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train_rfe, Y_train) 
Y_pred_rfr = rf.predict(X_test_rfe)

In [ ]:
print(evaluate_clf(Y_test, Y_pred_rfr))

(array([0.59457965, 0.60707269]), array([0.57333333, 0.62772981]), array([0.58376324, 0.61722846]), array([1875, 1969]))


# PCA Feature

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train_new, Y_train) 
Y_pred_rfp = rf.predict(X_test_new)

In [ ]:
c

(array([0.54277778, 0.56066536]), array([0.52106667, 0.58202133]), array([0.53170068, 0.57114378]), array([1875, 1969]))


# LASSO Feature

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train_lasso, Y_train) 
Y_pred_rfl = rf.predict(X_test_lasso)

In [ ]:
print(evaluate_clf(Y_test, Y_pred_rfp))